# ADA 2018 -  “Happiness” Share it through music.

#### Andres Montero, Ariel Alba, Diego Iriarte




In [ ]:
% matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import reverse_geocoder as rg
import os.path
import ast


In [ ]:
# Constants
DATA_DIR = './data/'
MUSIC_DIR = '{dir}{file}/'.format(dir=DATA_DIR, 
                                  file='fma_metadata')
PKL_DIR = '{dir}{file}/'.format(dir=DATA_DIR, 
                               file='pkl')

# True if we want to to execute the clean phase and to force 
# saving the cleaned file
CLEAN_PHASE = True


In [ ]:
if CLEAN_PHASE:
    echonest_path = '{dir}{file}'.format(dir=MUSIC_DIR, 
                                         file='echonest.csv')
    features_path = '{dir}{file}'.format(dir=MUSIC_DIR,
                                         file='features.csv')
    genres_path = '{dir}{file}'.format(dir=MUSIC_DIR,
                                       file='genres.csv')
    tracks_path = '{dir}{file}'.format(dir=MUSIC_DIR,
                                       file='tracks.csv')
    

In [ ]:
if CLEAN_PHASE:
    # Load datasets into pandas dataframes
    echonest_col_names=['track_id','danceability','energy',
                        'artist_latitude','artist_longitude',
                        'artist_name','artist_discovery',
                        'artist_family','artist_hotness',
                        'song_currency','song_hotness']
    
    echonest_dtypes = {'track_id': int, 'danceability': float, 
                       'energy': float, 'artist_latitude': float, 
                       'artist_longitude': float, 'artist_name': str,
                       'artist_discovery': float, 'artist_family': float,
                       'artist_hotness': float, 'song_currency': float,
                       'song_hotness': float}
    
    echonest_df = pd.read_csv(echonest_path, names=echonest_col_names,
                              header=3, dtype=echonest_dtypes, 
                              usecols=[0,2,3,11,13,14,21,22,23,24,25])

    genres_dtypes = {'genre_id': int, '#tracks': int, 
                     'parent': int, 'top_level': int}
    
    genres_df = pd.read_csv(genres_path, dtype=genres_dtypes)
    
    track_col_names = ['track_id', 'album_date_created',
                       'album_date_released', 'album_id',
                       'album_listens', 'album_title', 'artist_id',
                       'artist_name', 'track_duration', 'track_genre_top',
                       'track_language', 'track_listens', 'track_tags', 
                       'track_title']

    tracks_dtypes = {'track_id': int, 'album_date_created': str,
                     'album_date_released': str, 
                     'album_id': int, 'album_listens': int, 
                     'album_title': str, 'artist_id': int,
                     'artist_name': str, 'track_duration': int, 
                     'track_genre_top': str, 'track_language': str,
                     'track_listens': int, 'track_tags': str, 
                     'track_title': str}

    tracks_df = pd.read_csv(tracks_path, names=track_col_names,
                            header=2, usecols=[0, 2, 3, 6, 8, 11,
                                               21, 26, 38, 40, 45, 
                                               47, 51, 52])


In [ ]:
# Echonest clean phase 
if CLEAN_PHASE:
    # Get city, state, country from artist longitude and latitude
    cities=[]
    states=[]
    countries=[]
    
    for i in range (0, len(echonest_df)):
        if np.isnan(echonest_df.artist_latitude[i]):
            city = np.nan
            state = np.nan
            country = np.nan
        else:
            coordinates = (echonest_df.artist_latitude[i], 
                           echonest_df.artist_longitude[i])
            results = rg.search(coordinates, mode=1)
            city = results[0]['name']
            state = results[0]['admin1']
            country = results[0]['cc']
        
        cities.append(city)
        states.append(state)
        countries.append(country)
    
    echonest_df.insert(loc=5, column='city', 
                       value=pd.Series(cities))
    echonest_df.insert(loc=6, column='state',
                       value=pd.Series(states))
    echonest_df.insert(loc=7, column='country', 
                       value=pd.Series(countries))
    
    

In [ ]:
# Tracks clean phase
if CLEAN_PHASE:
    # Transforms str to datetime
    tracks_df['album_date_released'] = pd.to_datetime( \
                                            tracks_df['album_date_released'])
    tracks_df['album_date_created'] = pd.to_datetime( \
                                            tracks_df['album_date_created'])
    # Transform str to list
    tracks_df['track_tags'] = tracks_df['track_tags'].apply(lambda x: ast.literal_eval(x))


In [ ]:
# Read  and wirte files depending on the existance of own path
echonest_df_path = '{dir}{file}'.format(dir=PKL_DIR, 
                                        file='echonest_df.pkl')
genre_df_path = '{dir}{file}'.format(dir=PKL_DIR, 
                                     file='genres_df.pkl')
tracks_df_path = '{dir}{file}'.format(dir=PKL_DIR, 
                                      file='tracks_df.pkl')

if not CLEAN_PHASE or os.path.exists(echonest_df_path):
    echonest_df = pd.read_pickle(echonest_df_path)
else:
    echonest_df.to_pickle(echonest_df_path)
    
if not CLEAN_PHASE or os.path.exists(echonest_df_path):
    genres_df = pd.read_pickle(genre_df_path)
else:
    genres_df.to_pickle(genre_df_path)
    
if not CLEAN_PHASE or os.path.exists(tracks_df_path):
    tracks_df = pd.read_pickle(tracks_df_path)
else:
    tracks_df.to_pickle(tracks_df_path)
    

In [ ]:
print(len(echonest_df))
echonest_df.head()


In [ ]:
print(len(genres_df))
genres_df.head()


In [ ]:
print(len(tracks_df))
tracks_df.tail()
